In [ ]:
library(dplyr)
library(readr)
library(lubridate)
library(ggplot2)

# If you WANT elbow / silhouette plots later, install these and uncomment:
# install.packages(c("cluster", "factoextra"))
# library(cluster)
# library(factoextra)

# Data Sources (Vancouver Open Data)
urls <- c(
  "https://opendata.vancouver.ca/explore/dataset/business-licences/download/?format=csv",
  "https://opendata.vancouver.ca/explore/dataset/business-licences-2013-to-2024/download/?format=csv",
  "https://opendata.vancouver.ca/explore/dataset/business-licences-1997-to-2012/download/?format=csv"
)

# Function to load and clean each dataset
load_and_clean_data <- function(url) {
  df <- read_delim(url, delim = ";", show_col_types = FALSE)

  # Rename FOLDERYEAR -> Year if present
  if ("FOLDERYEAR" %in% names(df)) {
    df <- df %>% dplyr::rename(Year = FOLDERYEAR)
  }

  # Standardize names to lower case
  names(df) <- tolower(names(df))

  # Keep only the columns we need
  df <- df %>%
    dplyr::select(
      year,
      localarea,
      businessname,
      businesstype
    ) %>%
    dplyr::mutate(
      businessname = tolower(trimws(businessname)),
      businesstype = tolower(trimws(businesstype))
    )

  # Create an industry group variable from businesstype
  df <- df %>%
    dplyr::mutate(
      IndustryGroup = dplyr::case_when(
        grepl("retail|dealer|shop|store|market", businesstype, ignore.case = TRUE) ~ "Retail",
        grepl("computer|software|tech|data|consultant|web", businesstype, ignore.case = TRUE) ~ "Technology",
        grepl("restaurant|food|cafe|liquor|pub|club", businesstype, ignore.case = TRUE) ~ "Food & Bev",
        TRUE ~ "Other"
      )
    )

  df
}

# Load all datasets and combine
all_licences <- lapply(urls, load_and_clean_data) %>%
  bind_rows()

# Clean localarea
licences_filtered <- all_licences %>%
  filter(
    !is.na(localarea),
    localarea != "",
    tolower(localarea) != "other",
    tolower(localarea) != "not applicable"
  ) %>%
  mutate(localarea = tools::toTitleCase(tolower(localarea)))

# ---- Feature engineering per LocalArea ----
localarea_summary <- licences_filtered %>%
  group_by(localarea) %>%
  summarise(
    total_businesses = n(),
    total_retail_businesses = sum(IndustryGroup == "Retail"),
    .groups = "drop"
  ) %>%
  mutate(
    retail_density_proportion = total_retail_businesses / total_businesses
  )

# Keep areas with enough data
localarea_clustering_data <- localarea_summary %>%
  filter(
    total_businesses >= 10,
    !is.nan(retail_density_proportion)
  )

# Features for clustering
clustering_features <- localarea_clustering_data %>%
  select(total_retail_businesses, retail_density_proportion)

# Scale features
scaled_features <- scale(clustering_features)

# ---- K-means clustering ----
set.seed(123)
k_optimal <- 4  # choose number of clusters
kmeans_model <- kmeans(scaled_features, centers = k_optimal, nstart = 25)

# Add cluster labels back
localarea_clustering_data$cluster <- as.factor(kmeans_model$cluster)

# ---- Cluster summary ----
cluster_summary <- localarea_clustering_data %>%
  group_by(cluster) %>%
  summarise(
    num_localareas = n(),
    avg_total_businesses = mean(total_businesses),
    avg_total_retail_businesses = mean(total_retail_businesses),
    avg_retail_density_proportion = mean(retail_density_proportion),
    .groups = "drop"
  ) %>%
  arrange(avg_retail_density_proportion)

cat("\nCluster Summary:\n")
print(cluster_summary)

# ---- Visualisations ----
# Retail density by cluster
p1 <- ggplot(localarea_clustering_data,
             aes(x = cluster, y = retail_density_proportion, fill = cluster)) +
  geom_boxplot() +
  labs(
    title = "Retail Density Proportion by Business District Cluster",
    x = "Business District Cluster",
    y = "Retail Density Proportion"
  ) +
  theme_minimal()

print(p1)

# Total retail businesses by cluster
p2 <- ggplot(localarea_clustering_data,
             aes(x = cluster, y = total_retail_businesses, fill = cluster)) +
  geom_boxplot() +
  labs(
    title = "Total Retail Businesses by Business District Cluster",
    x = "Business District Cluster",
    y = "Total Retail Businesses"
  ) +
  theme_minimal()

print(p2)

# Top 10 LocalAreas in each cluster
cat("\nLocal Areas by Cluster (Top 10 per cluster):\n")
localarea_clustering_data %>%
  select(localarea, cluster, retail_density_proportion) %>%
  arrange(cluster, desc(retail_density_proportion)) %>%
  group_by(cluster) %>%
  slice_head(n = 10) %>%
  print(n = Inf)
